In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Get the data from walmart and monster with inteval of 30 minutes

walmart = yf.Ticker("WMT")
monster = yf.Ticker("MNST")

walmart_data = walmart.history(period="60d", interval="90m")
monster_data = monster.history(period="60d", interval="90m")

walmart_data.reset_index(inplace=True)
monster_data.reset_index(inplace=True)

print(walmart_data)
print(monster_data)

                     Datetime       Open       High        Low      Close  \
0   2024-02-21 09:30:00-05:00  58.333332  58.526669  57.693333  57.986668   
1   2024-02-21 11:00:00-05:00  57.985001  58.133331  57.799999  57.950001   
2   2024-02-21 12:30:00-05:00  57.955002  57.986668  57.753330  57.800831   
3   2024-02-21 14:00:00-05:00  57.799999  57.843334  57.600002  57.669998   
4   2024-02-21 15:30:00-05:00  57.671665  58.020000  57.669998  57.903336   
..                        ...        ...        ...        ...        ...   
295 2024-05-15 09:30:00-04:00  59.830002  59.939999  59.590000  59.799999   
296 2024-05-15 11:00:00-04:00  59.810001  59.875000  59.599998  59.715000   
297 2024-05-15 12:30:00-04:00  59.709999  59.779999  59.610001  59.775002   
298 2024-05-15 14:00:00-04:00  59.775002  59.794998  59.470001  59.685001   
299 2024-05-15 15:30:00-04:00  59.685001  59.840000  59.509998  59.830002   

      Volume  Dividends  Stock Splits  
0    4121963        0.0           0

In [3]:
# plot both tickers on go figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=walmart_data.index, y=walmart_data['Close'], name='walmart'))
fig.add_trace(go.Scatter(x=monster_data.index, y=monster_data['Close'], name='monster'))

fig.show()

In [4]:
print(walmart_data)

                     Datetime       Open       High        Low      Close  \
0   2024-02-21 09:30:00-05:00  58.333332  58.526669  57.693333  57.986668   
1   2024-02-21 11:00:00-05:00  57.985001  58.133331  57.799999  57.950001   
2   2024-02-21 12:30:00-05:00  57.955002  57.986668  57.753330  57.800831   
3   2024-02-21 14:00:00-05:00  57.799999  57.843334  57.600002  57.669998   
4   2024-02-21 15:30:00-05:00  57.671665  58.020000  57.669998  57.903336   
..                        ...        ...        ...        ...        ...   
295 2024-05-15 09:30:00-04:00  59.830002  59.939999  59.590000  59.799999   
296 2024-05-15 11:00:00-04:00  59.810001  59.875000  59.599998  59.715000   
297 2024-05-15 12:30:00-04:00  59.709999  59.779999  59.610001  59.775002   
298 2024-05-15 14:00:00-04:00  59.775002  59.794998  59.470001  59.685001   
299 2024-05-15 15:30:00-04:00  59.685001  59.840000  59.509998  59.830002   

      Volume  Dividends  Stock Splits  
0    4121963        0.0           0

In [5]:
# plot eith candle stick

fig = go.Figure(data=[go.Candlestick(x=walmart_data.index,
                open=walmart_data['Open'],
                high=walmart_data['High'],
                low=walmart_data['Low'],
                close=walmart_data['Close'])])

# plot moving average of 20 ticks and 80 ticks

fig.add_trace(go.Scatter(x=walmart_data.index, y=walmart_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=walmart_data.index, y=walmart_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()

In [6]:
# count the number of times the 5 ticks MA crosses the 20 ticks MA

walmart_data['5T_MA'] = walmart_data['Close'].rolling(window=5).mean()
walmart_data['20T_MA'] = walmart_data['Close'].rolling(window=20).mean()

walmart_data['signal'] = np.where(walmart_data['5T_MA'] > walmart_data['20T_MA'], 1, 0)
walmart_data['signal'] = walmart_data['signal'].diff()

print(walmart_data['signal'].value_counts())

# plot the buy and sell signals

fig = go.Figure(data=[go.Candlestick(x=walmart_data.index,
                open=walmart_data['Open'],
                high=walmart_data['High'],
                low=walmart_data['Low'],
                close=walmart_data['Close'])])

fig.add_trace(go.Scatter(x=walmart_data.index, y=walmart_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=walmart_data.index, y=walmart_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = walmart_data[walmart_data['signal'] == 1]
sell_signals = walmart_data[walmart_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))

fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

signal
 0.0    277
 1.0     11
-1.0     11
Name: count, dtype: int64


In [7]:
# calculate profit and loss

walmart_data['buy_price'] = np.where(walmart_data['signal'] == 1, walmart_data['Close'], np.nan)
walmart_data['sell_price'] = np.where(walmart_data['signal'] == -1, walmart_data['Close'], np.nan)

walmart_data['buy_price'] = walmart_data['buy_price'].ffill()
walmart_data['sell_price'] = walmart_data['sell_price'].ffill()

walmart_data['PnL'] = walmart_data['sell_price'] - walmart_data['buy_price']

print(walmart_data['PnL'].sum())

# plot the profit and loss

fig = go.Figure(data=[go.Candlestick(x=walmart_data.index,
                open=walmart_data['Open'],
                high=walmart_data['High'],
                low=walmart_data['Low'],
                close=walmart_data['Close'])])

fig.add_trace(go.Scatter(x=walmart_data.index, y=walmart_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=walmart_data.index, y=walmart_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = walmart_data[walmart_data['signal'] == 1]
sell_signals = walmart_data[walmart_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

-85.08653259277344


In [8]:
ratio_0 = (walmart_data['Close'].iloc[0]) / (monster_data['Close'].iloc[0])
ratio_0

1.0533454756892908

In [9]:
monster_data['Close_NORM'] = monster_data['Close'].multiply(ratio_0).round(6)

In [10]:
walmart_monster_dif = walmart_data['Close'] - monster_data['Close_NORM']
walmart_monster_dif

0     -3.669434e-07
1     -1.420042e-01
2     -1.226382e-01
3     -3.114018e-01
4     -1.781334e-01
           ...     
295    2.566474e+00
296    2.560476e+00
297    2.351874e+00
298    2.325071e+00
299    2.696542e+00
Length: 300, dtype: float64

In [11]:
# Plot the difference between the two tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=walmart_monster_dif.index, y=walmart_monster_dif, name='walmart-monster'))

fig.show()

In [12]:
# Plot the two normalized tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=walmart_data.index, y=walmart_data['Close'], name='walmart'))
fig.add_trace(go.Scatter(x=monster_data.index, y=monster_data['Close_NORM'], name='monster'))

fig.show()

In [13]:
# Plot the difference and its moving average

fig = go.Figure()
fig.add_trace(go.Scatter(x=walmart_monster_dif.index, y=walmart_monster_dif, name='walmart-monster'))
fig.add_trace(go.Scatter(x=walmart_monster_dif.index, y=walmart_monster_dif.rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=walmart_monster_dif.index, y=walmart_monster_dif.rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()


In [14]:
walmart_monster_dif = walmart_monster_dif.to_frame()
walmart_monster_dif.columns = ['Close']

In [15]:
walmart_monster_dif['10T_MA'] = walmart_monster_dif['Close'].rolling(window=10).mean()
walmart_monster_dif['60T_MA'] = walmart_monster_dif['Close'].rolling(window=60).mean()

walmart_monster_dif['signal'] = np.where(walmart_monster_dif['10T_MA'] > walmart_monster_dif['60T_MA'], 1, 0)
walmart_monster_dif['signal'] = walmart_monster_dif['signal'].diff()

print(walmart_monster_dif['signal'].value_counts())

# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=walmart_monster_dif.index, y=walmart_monster_dif['Close'], name='walmart-monster'))
fig.add_trace(go.Scatter(x=walmart_monster_dif.index, y=walmart_monster_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
fig.add_trace(go.Scatter(x=walmart_monster_dif.index, y=walmart_monster_dif.rolling(window=50).mean()['Close'], name='60 Ticks MA'))

buy_signals = walmart_monster_dif[walmart_monster_dif['signal'] == -1]
sell_signals = walmart_monster_dif[walmart_monster_dif['signal'] == 1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

walmart_data['signal'] = walmart_monster_dif['signal']
monster_data['signal'] = - walmart_monster_dif['signal'] # opposite signal

# Calculate the profit and loss

walmart_data['buy_price'] = np.where(walmart_data['signal'] == 1, walmart_data['Close'], np.nan)
walmart_data['sell_price'] = np.where(walmart_data['signal'] == -1, walmart_data['Close'], np.nan)

monster_data['signal']

monster_data['buy_price'] = np.where(monster_data['signal'] == 1, monster_data['Close_NORM'], np.nan)
monster_data['sell_price'] = np.where(monster_data['signal'] == -1, monster_data['Close_NORM'], np.nan)

walmart_profit = walmart_data['sell_price'].sum() - walmart_data['buy_price'].sum()
monster_profit = monster_data['sell_price'].sum() - monster_data['buy_price'].sum()

print('WMT Profit => ', walmart_profit)
print('MNST Profit => ', monster_profit)

pair_tsm_dis_profit = walmart_profit + monster_profit

print('Pair profit => ', pair_tsm_dis_profit)


signal
 0.0    293
 1.0      3
-1.0      3
Name: count, dtype: int64


WMT Profit =>  -2.549999237060547
MNST Profit =>  3.902641999999986
Pair profit =>  1.352642762939439


In [16]:
#Consider walmart as the first and monster as the second ticker.
#When there is a buy signal for walmart, we buy walmart and sell monster.
#When there is a sell signal for walmart, we sell walmart and buy monster.

In [17]:
from statsmodels.tsa.stattools import coint

# Assuming you have already imported the necessary data for Walmart and Monster Beverage
# Let's say walmart_data['Date'] represents the datetime column for Walmart data, 
# and monster_data['Date'] represents the datetime column for Monster Beverage data

# Convert datetime columns to indexes
walmart_data.set_index('Datetime', inplace=True)
monster_data.set_index('Datetime', inplace=True)

# Find common datetime values
common_dates = walmart_data.index.intersection(monster_data.index)

# Align the data and handle missing values
walmart_close_aligned = walmart_data.loc[common_dates, 'Close']
monster_close_aligned = monster_data.loc[common_dates, 'Close_NORM']

# Perform cointegration test
cointegration_test_result = coint(walmart_close_aligned, monster_close_aligned)

# Print the result
print("Cointegration Test Result:")
print("Test statistic:", cointegration_test_result[0])
print("p-value:", cointegration_test_result[1])
print("Critical values:", cointegration_test_result[2])

Cointegration Test Result:
Test statistic: -3.845760132083532
p-value: 0.011766241536262023
Critical values: [-3.93344345 -3.35664144 -3.05866504]


In [18]:
monster_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Close_NORM,signal,buy_price,sell_price
Datetime,,,,,,,,,,,
2024-02-21 09:30:00-05:00,55.439999,55.660000,54.904999,55.049999,1410039,0.0,0.0,57.986668,NaN,NaN,NaN
2024-02-21 11:00:00-05:00,55.049999,55.240002,54.915001,55.150002,1107706,0.0,0.0,58.092005,-0.0,NaN,NaN
2024-02-21 12:30:00-05:00,55.145000,55.174999,54.980000,54.990002,564317,0.0,0.0,57.923469,-0.0,NaN,NaN
2024-02-21 14:00:00-05:00,54.990002,55.180000,54.965000,55.044998,954771,0.0,0.0,57.981400,-0.0,NaN,NaN
2024-02-21 15:30:00-05:00,55.044998,55.180000,55.040001,55.139999,869793,0.0,0.0,58.081469,-0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-15 09:30:00-04:00,54.509998,54.625000,54.305000,54.334999,930925,0.0,0.0,57.233525,-0.0,NaN,NaN
2024-05-15 11:00:00-04:00,54.345001,54.424999,54.119999,54.259998,762919,0.0,0.0,57.154524,-0.0,NaN,NaN
2024-05-15 12:30:00-04:00,54.264999,54.555000,54.165001,54.514999,638217,0.0,0.0,57.423128,-0.0,NaN,NaN
